<image src="https://raw.githubusercontent.com/ramiro999/pytorch-exploration/main/images/Banner-NiN.png" width=100%>


# <font color='#4C5FDA'> **Network in Network** </font>

El articulo <font color="EB9A54">"Network In Network"</font> propone una arquitectura avanzada que mejora las capacidades de las redes neuronales convolucionales (CNN) tradicionales mediante el uso de microrredes neuronales, concretamente perceptrones multicapa (MLP), dentro de cada capa convolucional. Esta estructura permite una representación más abstracta de los datos en cada capa, mejorando potencialmente el rendimiento de la clasificación.

<image src="https://raw.githubusercontent.com/ramiro999/pytorch-exploration/main/images/NiN-1.png" >




<image src="https://raw.githubusercontent.com/ramiro999/pytorch-exploration/main/images/NiN-2.png" >

<font color="EB9A54">Capa de Convolución MLP (mlpconv): </font> Esta capa sustituye la convolución lineal estándar en las CNNs por un mini MLP que procesa cada parche de la imagen de entrada. Es una combinación de múltiples capas totalmente conectadas con activaciones ReLU.

<font color="EB9A54"> Global Average Pooling (GAP): </font> en lugar de utilizar capas totalmente conectadas en la parte superior de la red, la NIN utiliza una capa de agrupación de promedios globales seguida de una activación softmax para la clasificación. Esto reduce el número total de parámetros y ayuda a reducir el sobreajuste.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MLPConv(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
    super(MLPConv, self).__init__()
    # Las 3 capas definidas
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
    self.fc1 = nn.Conv2d(out_channels, out_channels, 1)
    self.fc2 = nn.Conv2d(out_channels, out_channels, 1)


  # El metodo forward de la clase define como los datos de entrada son procesados a traves de la red.
  def forward(self, x):
    x = self.conv(x)
    x = F.relu(x)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return x

In [ ]:
class NIN(nn.Module):
  def __init__(self, num_classes=10):
    super(NIN, self).__init__()
    # Primer capa MLPConv: procesa la imagen de entrada
    self.mlpconv1 = MLPConv(3, 192, kernel_size=5, padding=2)
    # Segunda capa MLPConv: procesamiento mas profundo de caracteristicas
    self.mlpconv2 = MLPConv(192, 160, kernel_size=5, padding=2)
    # Procesamiento mas profundo antes de reducir la resolución.
    self.mlpconv3 = MLPConv(160, 96, kernel_size=5, padding=2)
    # Capa de agrupacion promedio para reducir las dimensiones espaciales
    self.pooling = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
    # Capas MLPConv adicionales despues de la reduccion de resolucion
    self.mlpconv4 = MLPConv(96, 192, kernel_size=3, padding=1)
    # Convoluciones 1x1 que actuan antes de combinar las caracteristicas antes de la clasificación final.
    self.mlpconv5 = MLPConv(192, 192, kernel_size=1)
    self.mlpconv6 = MLPConv(192, 10, kernel_size=1)
    # Agrupamiento promedio global para agregar datos espaciales.
    self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

  def forward(self,x):
    # Pase sequencialmente a traves de las 3 primeras capas con agrupamiento.
    x = self.mlpconv1(x)
    x = self.pooling(x)
    x = self.mlpconv2(x)
    x = self.pooling(x)
    x = self.mlpconv3(x)
    x = self.pooling(x)
    # Capas adicionales sin agrupacion intermedia.
    x = self.mlpconv4(x)
    x = self.mlpconv5(x)
    x = self.mlpconv6(x)
    # Aplicar agrupamiento global promedio para preparar la clasificacion.
    x = self.global_avg_pool(x)
    # Aplanar la salida para la capa de clasificacion final.
    x = x.view(x.size(0), -1)
    return x

model = NIN(num_classes=10)


---
# <font color='#4C5FDA'> **Referencias** </font>

Network In Network

https://arxiv.org/pdf/1312.4400

---

**Elaborado por Ramiro Santiago Avila Chacon**